# **Estimate CO2 emissions from lakes and reservoirs**

In [19]:
# @Date    : 2025-07-30
# @Author  : Li Yuxin (leeeyuxin@connect.hku.hk)
# @Link    : https://github.com/liiiyuxin/Lentic_CO2.git
# @Version : 3.13.5

# <span style="color:blue"> *Setups* </span>

In [2]:
## env--rf
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm

import glob
import os
import scipy.io
import math
import calendar
import matplotlib as mpl

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from PIL import Image
from osgeo import gdal
from matplotlib import cm
from matplotlib.patches import Patch
from geopy.distance import geodesic

import rasterio
from rasterio.transform import from_origin
from rasterio.transform import rowcol
import contextily as ctx

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from itertools import combinations
from scipy.stats import gaussian_kde, pearsonr

import xgboost as xgb
import shap

# <span style="color:blue"> *Function* </span>

In [4]:
def POWER(a, b):
    return a ** b
    
def cal_pCO2water(TAlk, WT, pH):
    # calculate the pCO2water
    # ref: Sharp, J. et al. CO2SYSv3 for MATLAB. Zenodo. doi 10, (2021)
    
    # -- input:
    # TAlk: total alkalinity [µeq/L]
    # WT: water tempreture [°C]
    # pH [/]
    
    # -- output:
    # pCO2water: surface water CO2 partial pressure [μatm]
    
    T6 = pH
    U6 = WT
    V6 = TAlk
    pCO2water = round(((V6-((POWER(10,-(0.00009*POWER(U6,2)-0.0137*U6+10.62)))*V6/(2*(POWER(10,-(0.00009*POWER(U6,2)-0.0137*U6+10.62)))+POWER(10,-T6))))*POWER(10,-T6)/POWER(10,-(0.00011*POWER(U6,2)-0.012*U6+6.58)))/(POWER(10,-(-0.00007*POWER(U6,2)+0.016*U6+1.11))),0)
    return pCO2water

In [5]:
def cal_k(WT, WS):
    # calculate the gas transfer velocity in lakes and reservoirs
    # ref1: Wanninkhof, R. Relationship between wind speed and gas exchange over the ocean revisited. Limnol. Oceanogr. Methods 12, 351–362 (2014).
    # ref2: Cole, J. J. & Caraco, N. F. Atmospheric exchange of carbon dioxide in a low‐wind oligotrophic lake measured by the addition of SF6. Limnol. Oceanogr. Methods 43, 647–656 (1998).
    
    # -- input:
    # k: Gas transfer velocity [cm/hr]
    # WT: Water temperature [°C]
    # WS: Wind Speed at 10m [m/s]

    # -- output:
    # Sc: Schmidt number [/]
    
    WT = np.asarray(WT)
    WS = np.asarray(WS)
    k = np.zeros_like(WT)

    # calculate Sc
    Sc = 1923.6 - 125.06 * WT + 4.3773 * WT ** 2 - 0.085681 * WT ** 3 + 0.00070284 * WT ** 4
    
    # calculate k under different wind speed levels
    k[WS < 3.7] = (2.07 + 0.215 * WS[WS < 3.7] ** 1.7) * (Sc[WS < 3.7] / 600) ** -2/3
    k[WS >= 3.7] = 0.251 * WS[WS >= 3.7] ** 2 * (Sc[WS >= 3.7] / 600) ** -1/2

    return k

In [6]:
def cal_kH(WT):
    # calculate Henry’s constant for CO2 corrected for temperature in lakes and reservoirs
    # ref: Weiss, R. F. Carbon dioxide in water and seawater: the solubility of a non-ideal gas. Mar. Chem. 2, 203–215 (1974).

    # -- input:
    # WT: Water temperature [K]

    # -- output:
    # kH: Henry constant [mol L-1 atm-1]
    
    WT = np.asarray(WT)
    kH = np.full(WT.shape, np.nan)
    valid_indices = ~np.isnan(WT)
    kH[valid_indices] = np.exp(-58.0931 + 90.5069 * 100 / WT[valid_indices] + 22.294 * np.log(WT[valid_indices] / 100))
    
    return kH

# <span style="color:red"> *Modelling of total alkalinity* </span>

In [7]:
##### read data

path_GLORICH = '/Users/leee/OneDrive/HKU/Data/Glorich_V01_CSV_plus_Shapefiles_2019_05_24/hydrochemistry.csv'
# DataSource: https://doi.pangaea.de/10.1594/PANGAEA.902360

df_glorich = pd.read_csv(path_GLORICH, encoding='GBK')
df_glorich = df_glorich[['STAT_ID','Temp_water', 'pH', 'DO_mgL', 'SpecCond25C', 'TN', 'TP', 'Alkalinity']]
df_glorich.rename(columns={'Temp_water': 'WT', 'DO_mgL': 'DO', 'SpecCond25C': 'EC', 'Alkalinity': 'TAlk'}, inplace=True)
df_glorich = df_glorich.dropna(axis=0).reset_index(drop=True) # Drop missing values
df_glorich = df_glorich[['WT', 'pH', 'DO', 'EC', 'TN', 'TP', 'TAlk']]

# Convert the unit of TN & TP from µmol/L to mg/L
df_glorich['TN'] = df_glorich['TN']/ 1000 * 14.007
df_glorich['TP'] = df_glorich['TP']/ 1000 * 30.974

In [8]:
##### Comparison of the performance of the two machine learning models in the prediction of total alkalinity

### Define feature variables and target variable
X = df_glorich[['WT', 'pH', 'DO', 'EC', 'TN', 'TP']]
y = df_glorich['TAlk']

### Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

In [9]:
### Fit the RF_BAT model to training data
model_rfbat = RandomForestRegressor()
model_rfbat.fit(X_train, y_train)

# Predict target values for test data
y_pred = model_rfbat.predict(X_test)

# Calculate performance metrics
model_lr = LinearRegression()
model_lr.fit(y_test.values.reshape(-1, 1), y_pred)
slope = model_lr.coef_[0]
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
nrmse = rmse / np.std(y_pred)
mre = np.mean(((y_test - y_pred) / y_test)) * 100

# Output model performance metrics
print('Model performance of RF-BAT:')
print(f'- Slope: {slope}')
print(f'- R2: {r2}')
print(f'- NRMSE: {nrmse}')
print(f'- MRE(%): {mre}')

Model performance of RF-BAT:
- Slope: 0.7488218308549538
- R2: 0.7728785565409519
- NRMSE: 0.5597979757185385
- MRE(%): -8.439821177841187


In [10]:
### Fit the XGBoost model to training data
model_xgb = xgb.XGBRegressor()
model_xgb.fit(X_train, y_train)

# Predict target values for test data
y_pred = model_xgb.predict(X_test)

# Calculate performance metrics
model_lr = LinearRegression()
model_lr.fit(y_test.values.reshape(-1, 1), y_pred)
slope = model_lr.coef_[0]
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
nrmse = rmse / np.std(y_pred)
mre = np.mean(((y_test - y_pred) / y_test)) * 100

# Output model performance metrics
print('Model performance of XGBoost:')
print(f'- Slope: {slope}')
print(f'- R2: {r2}')
print(f'- NRMSE: {nrmse}')
print(f'- MRE(%): {mre}')

Model performance of XGBoost:
- Slope: 0.7730325927055115
- R2: 0.7733532676372779
- NRMSE: 0.5415894986663005
- MRE(%): -7.737216116988645


In [11]:
##### Identify the top five predictor groups in the prediction of total alkalinity for XGBoost
import itertools
predictors = ['WT', 'pH', 'DO', 'EC', 'TN', 'TP']
predictor_groups = []
scores = []

for r in range(1, len(predictors)+1):
    for comb in itertools.combinations(predictors, r):
        X = df_glorich[list(comb)]
        y = df_glorich['TAlk']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

        model = xgb.XGBRegressor()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        model_lr = LinearRegression()
        model_lr.fit(y_test.values.reshape(-1, 1), y_pred)
        slope = model_lr.coef_[0]
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
        nrmse = rmse/np.std(y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        mre = np.mean(((y_test - y_pred) / y_test)) * 100

        predictor_groups.append(list(comb))
        scores.append((slope, r2, nrmse, mre))

results = pd.DataFrame({'Predictor groups': predictor_groups, 'Slope': [score[0] for score in scores],
                       'R2': [score[1] for score in scores], 'NRMSE': [score[2] for score in scores], 
                       'MRE(%)': [score[3] for score in scores]})
top5_results = results.sort_values(by='R2', ascending=False, ignore_index=True).head(5)
del results, predictor_groups, scores
print(top5_results)

           Predictor groups     Slope        R2     NRMSE    MRE(%)
0  [WT, pH, DO, EC, TN, TP]  0.773033  0.773353  0.541589 -7.737216
1      [WT, pH, EC, TN, TP]  0.769237  0.765587  0.550723 -8.178742
2      [pH, DO, EC, TN, TP]  0.766761  0.757047  0.559370 -8.245745
3      [WT, pH, DO, EC, TN]  0.762257  0.756667  0.562945 -8.593666
4          [pH, EC, TN, TP]  0.754356  0.749372  0.574524 -7.968240


# <span style="color:red"> *Estimation of pCO2water* </span>

In [12]:
##### Read water chemistry data
df = pd.read_csv('/Users/leee/OneDrive/HKU/Project/CO2_china/Code/input_data/water_chemistry_example.txt', sep='\t')

##### Estimate WT with t2m
# ref: Mohseni, O., Stefan, H. G. & Erickson, T. R. A nonlinear regression model for weekly stream temperatures. Water Resour. Res. 34, 2685–2692 (1998)
# WT: Water temperature [°C]
# t2m: Air temperature at 2 m height [°C]
df['WT'] = 0.8 + (26.2 - 0.8)/(1 + np.exp( 0.18 * (13.3 - df['t2m'])))

##### Estimate TAlk with WT, pH, DO, EC, TN and TP
df['TAlk'] = model_xgb.predict(df[['WT', 'pH', 'DO', 'EC', 'TN', 'TP']])

##### Estimate pCO2water with TAlk, WT and pH
df['pCO2_water'] = cal_pCO2water(df['TAlk'], df['WT'], df['pH'])

# <span style="color:red"> Estimation of areal CO2 emission rate</span>

In [13]:
# Estimate gas transfer velocity
# k: gas transfer velocity [cm h−1]
df['k'] = cal_k(df['WT'].values, df['si10'].values)

# Estimate Henry’s constant for CO2
# kH: Henry’s constant for CO2 [/]
df['kH'] = cal_kH(df['WT'].values + 273.15)

# Calculate areal CO2 emission rate
# F_CO2: Areal CO2 emission rate [g CO2 m-2 d-1]
df['F_CO2'] = df['k'] * df['kH'] * (df['pCO2_water'] - df['pCO2_air']) * 44 /1000

# <span style="color:red"> Estimation of total CO2 efflux </span>

In [14]:
#### Get number of days in the year
df['n_days'] = df['year'].apply(lambda y: 366 if calendar.isleap(y) else 365)

#### Calculate total CO2 efflux
# E_CO2: Total CO2 emissions [Tg CO2 yr-1]
# Area: Water surface area [km2]
df['E_CO2'] = (df['F_CO2']) * (df['Area'] * 10**6) * df['n_days'] / 10**12

# <span style="color:red"> Identify the driving factors of FCO2</span>

In [15]:
### Define feature categories
meteorological_factors = ['t2m', 'precip', 'si10', 'ssrd', 'pCO2_air']
soil_properties_factors = ['cSoil', 'Soil_TP', 'Soil_TN', 'Soil_PH']
terrestrial_productivity_factors = ['NDVI', 'gpp_mod', 'npp_mod']
human_activity_factors = ['pop', 'N_total', 'Cropland']
water_chemistry_factors = ['pH', 'DO', 'EC', 'Tur', 'NH4-N', 'TN', 'TP', 'DOC']

factors_dict = {
    'Meteorology': meteorological_factors,
    'Soil Properties': soil_properties_factors,
    'Human Activity': human_activity_factors,
    'Terrestrial Productivity': terrestrial_productivity_factors,
    'Water Chemistry': water_chemistry_factors
}

In [16]:
### Select the specified columns from the dataframe
columns_k = ['F_CO2']+ meteorological_factors + soil_properties_factors + terrestrial_productivity_factors + human_activity_factors +water_chemistry_factors
ddf = df[columns_k]

### Separate features (X) and target variable (y)
X = ddf.drop(columns='F_CO2')
y = ddf['F_CO2']

### Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Fit the XGBoost model to training data
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model.fit(X_train, y_train)

### Predict target values for test data
y_pred = model.predict(X_test)

# Calculate performance metrics
model_lr = LinearRegression()
model_lr.fit(y_test.values.reshape(-1, 1), y_pred)
slope = model_lr.coef_[0]
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
nrmse = rmse / np.std(y_pred)
mre = np.mean(((y_test - y_pred) / y_test)) * 100

# Output model performance metrics
print('Model performance:')
print(f'- Slope: {slope}')
print(f'- R2: {r2}')
print(f'- NRMSE: {nrmse}')
print(f'- MRE(%): {mre}')

Model performance:
- Slope: 0.9404485949306894
- R2: 0.909523661078186
- NRMSE: 0.3052151812114331
- MRE(%): -3.1557090356016033


In [18]:
### Initialize SHAP TreeExplainer for the trained model
explainer = shap.TreeExplainer(model)

### Calculate SHAP values for the test features
shap_values = explainer.shap_values(X_test)

### Compute the mean absolute SHAP value for each feature to assess feature importance
importances = np.abs(shap_values).mean(axis=0)
df_mean_shap = pd.DataFrame({'Index': list(X_test.columns), 'SHAP': importances})
print(df_mean_shap)

       Index      SHAP
0        t2m  0.490113
1     precip  0.008154
2       si10  0.655974
3       ssrd  0.015712
4   pCO2_air  0.063855
5      cSoil  0.011796
6    Soil_TP  0.011149
7    Soil_TN  0.011785
8    Soil_PH  0.011620
9       NDVI  0.006226
10   gpp_mod  0.009011
11   npp_mod  0.007751
12       pop  0.012467
13   N_total  0.024751
14  Cropland  0.048810
15        pH  0.347156
16        DO  0.048997
17        EC  0.053984
18       Tur  0.115934
19     NH4-N  0.008767
20        TN  0.019041
21        TP  0.029118
22       DOC  0.012966
